# Feature Engineering

---

1. Import packages
2. Load data
3. Feature engineering

---

## 1. Import packages

In [1]:
import pandas as pd

---
## 2. Load data

In [3]:
df = pd.read_csv('./clean_data_after_eda.csv')
df["date_activ"] = pd.to_datetime(df["date_activ"], format='%Y-%m-%d')
df["date_end"] = pd.to_datetime(df["date_end"], format='%Y-%m-%d')
df["date_modif_prod"] = pd.to_datetime(df["date_modif_prod"], format='%Y-%m-%d')
df["date_renewal"] = pd.to_datetime(df["date_renewal"], format='%Y-%m-%d')

In [5]:
df.head(3)

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,var_6m_price_off_peak_var,var_6m_price_peak_var,var_6m_price_mid_peak_var,var_6m_price_off_peak_fix,var_6m_price_peak_fix,var_6m_price_mid_peak_fix,var_6m_price_off_peak,var_6m_price_peak,var_6m_price_mid_peak,churn
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,...,0.000131,4.100838e-05,0.000908,2.086294,99.530517,44.235794,2.086425,9.953056e+01,44.236702,1
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,0.000003,1.217891e-03,0.000000,0.009482,0.000000,0.000000,0.009485,1.217891e-03,0.000000,0
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,...,0.000004,9.450150e-08,0.000000,0.000000,0.000000,0.000000,0.000004,9.450150e-08,0.000000,0


---

## 3. Feature engineering

### Difference between off-peak prices in December and preceding January

Below is the code created by your colleague to calculate the feature described above. Use this code to re-create this feature and then think about ways to build on this feature to create features with a higher predictive power.

In [9]:
price_df = pd.read_csv('price_data.csv')
price_df["price_date"] = pd.to_datetime(price_df["price_date"], format='%Y-%m-%d')
price_df.head()

,id,price_date,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix
0,038af19179925da21a25619c5a24b745,2015-01-01,0.151367,0.0,0.0,44.266931,0.0,0.0
1,038af19179925da21a25619c5a24b745,2015-02-01,0.151367,0.0,0.0,44.266931,0.0,0.0
2,038af19179925da21a25619c5a24b745,2015-03-01,0.151367,0.0,0.0,44.266931,0.0,0.0
3,038af19179925da21a25619c5a24b745,2015-04-01,0.149626,0.0,0.0,44.266931,0.0,0.0
4,038af19179925da21a25619c5a24b745,2015-05-01,0.149626,0.0,0.0,44.266931,0.0,0.0


In [11]:
# Group off-peak prices by companies and month
monthly_price_by_id = price_df.groupby(['id', 'price_date']).agg({'price_off_peak_var': 'mean', 'price_off_peak_fix': 'mean'}).reset_index()

# Get january and december prices
jan_prices = monthly_price_by_id.groupby('id').first().reset_index()
dec_prices = monthly_price_by_id.groupby('id').last().reset_index()

# Calculate the difference
diff = pd.merge(dec_prices.rename(columns={'price_off_peak_var': 'dec_1', 'price_off_peak_fix': 'dec_2'}), jan_prices.drop(columns='price_date'), on='id')
diff['offpeak_diff_dec_january_energy'] = diff['dec_1'] - diff['price_off_peak_var']
diff['offpeak_diff_dec_january_power'] = diff['dec_2'] - diff['price_off_peak_fix']
diff = diff[['id', 'offpeak_diff_dec_january_energy','offpeak_diff_dec_january_power']]
diff.head()

,id,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power
0,0002203ffbb812588b632b9e628cc38d,-0.006192,0.162916
1,0004351ebdd665e6ee664792efc4fd13,-0.004104,0.177779
2,0010bcc39e42b3c2131ed2ce55246e3c,0.050443,1.500000
3,0010ee3855fdea87602a5b7aba8e42de,-0.010018,0.162916
4,00114d74e963e47177db89bc70108537,-0.003994,-0.000001


Now it is time to get creative and to conduct some of your own feature engineering! Have fun with it, explore different ideas and try to create as many as yo can!

In [13]:
df['tenure_days'] = (df['date_end'] - df['date_activ']).dt.days

In [15]:
df['days_since_renewal'] = (df['date_end'] - df['date_renewal']).dt.days

In [17]:
df['days_since_modif_prod'] = (df['date_end'] - df['date_modif_prod']).dt.days

In [19]:
monthly_price_by_id = price_df.groupby(['id', 'price_date']).agg({'price_off_peak_var': 'mean'}).reset_index()
price_trend = monthly_price_by_id.groupby('id').agg(
    price_change=('price_off_peak_var', lambda x: x.iloc[-1] - x.iloc[0]),
    price_std=('price_off_peak_var', 'std')
).reset_index()

In [21]:
price_df['peak_to_offpeak_ratio'] = price_df['price_peak_var'] / (price_df['price_off_peak_var'] + 1e-6)  # Avoid division by zero
avg_ratio = price_df.groupby('id')['peak_to_offpeak_ratio'].mean().reset_index()

In [23]:
df['cons_change'] = df['cons_12m'] - 12 * df['cons_last_month']

In [25]:
df['gas_to_elec_ratio'] = df['cons_gas_12m'] / (df['cons_12m'] + 1e-6)

In [27]:
df['forecast_accuracy'] = df['cons_12m'] - df['forecast_cons_12m']

In [29]:
df_merged = df.merge(price_df.groupby('id').agg({'price_off_peak_var': 'mean', 'price_off_peak_fix': 'mean'}).reset_index(), on='id')
df_merged['est_annual_cost'] = df_merged['cons_12m'] * df_merged['price_off_peak_var'] + df_merged['price_off_peak_fix'] * 12

In [31]:
df_merged = df_merged.merge(diff, on='id')
df_merged['cost_impact_energy'] = df_merged['offpeak_diff_dec_january_energy'] * df_merged['cons_12m']

In [33]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['channel_sales_encoded'] = le.fit_transform(df['channel_sales'])

In [35]:
df['has_gas'] = (df['cons_gas_12m'] > 0).astype(int)

In [37]:
df_merged['price_change_tenure'] = df_merged['offpeak_diff_dec_january_energy'] * df_merged['tenure_days']

In [39]:
df_merged = df_merged.merge(price_trend, on='id')
df_merged['cons_price_volatility'] = df_merged['cons_12m'] * df_merged['price_std']

In [41]:
monthly_price_by_id['price_diff'] = monthly_price_by_id.groupby('id')['price_off_peak_var'].diff()
max_price_increase = monthly_price_by_id.groupby('id')['price_diff'].max().reset_index(name='max_price_increase')

In [43]:
df_merged

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,price_off_peak_var,price_off_peak_fix,est_annual_cost,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power,cost_impact_energy,price_change_tenure,price_change,price_std,cons_price_volatility
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,...,0.124787,40.942265,491.307186,0.020057,3.700961,0.000000,21.982472,0.020057,0.007829,0.000000
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,0.149609,44.311375,1228.914055,-0.003767,0.177779,-17.554220,-9.666122,-0.003767,0.002212,10.309758
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,...,0.170512,44.385450,625.383751,-0.004670,0.177779,-2.540480,-10.236640,-0.004670,0.002396,1.303155
3,bba03439a292a1e166f80264c16191cb,lmkebamcaaclubfxadlmueccxoimlema,1584,0,0,2010-03-30,2016-03-30,2010-03-30,2015-03-31,240.04,...,0.151210,44.400265,772.319955,-0.004547,0.177779,-7.202448,-9.967024,-0.004547,0.002317,3.669515
4,149d57cf92fc41cf94415803a877cb4b,MISSING,4425,0,526,2010-01-13,2016-03-07,2010-01-13,2015-03-09,445.75,...,0.124174,40.688156,1037.729298,-0.006192,0.162916,-27.399600,-13.901040,-0.006192,0.003847,17.023641
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14601,18463073fb097fc0ac5d3e040f356987,foosdfpfkusacimwkcsosbicdxkicaua,32270,47940,0,2012-05-24,2016-05-08,2015-05-08,2014-05-26,4648.01,...,0.144124,44.370635,5183.342550,-0.008653,0.177779,-279.232310,-12.503585,-0.008653,0.003897,125.741172
14602,d0a6f71671571ed83b2645d23af6de00,foosdfpfkusacimwkcsosbicdxkicaua,7223,0,181,2012-08-27,2016-08-27,2012-08-27,2015-08-28,631.69,...,0.106799,59.015674,1479.597861,-0.007395,0.236694,-53.414085,-10.804095,-0.007395,0.003669,26.498180
14603,10e6828ddd62cbcf687cb74928c4c2d2,foosdfpfkusacimwkcsosbicdxkicaua,1844,0,179,2012-02-08,2016-02-07,2012-02-08,2015-02-09,190.39,...,0.124338,40.701732,717.700829,-0.006192,0.162916,-11.418048,-9.040320,-0.006192,0.003976,7.332612
14604,1cf20fd6206d7678d5bcafd28c53b4db,foosdfpfkusacimwkcsosbicdxkicaua,131,0,0,2012-08-30,2016-08-30,2012-08-30,2015-08-31,19.34,...,0.149609,44.311375,551.335272,-0.003767,0.177779,-0.493477,-5.503587,-0.003767,0.002212,0.289824


In [47]:
df

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,var_6m_price_peak,var_6m_price_mid_peak,churn,tenure_days,days_since_renewal,days_since_modif_prod,cons_change,gas_to_elec_ratio,forecast_accuracy,channel_sales_encoded
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,...,9.953056e+01,4.423670e+01,1,1096,358,227,0,5.494600e+10,0.00,4
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,1.217891e-03,0.000000e+00,0,2566,365,2566,4660,0.000000e+00,4470.05,0
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,...,9.450150e-08,0.000000e+00,0,2192,365,2192,544,0.000000e+00,496.04,4
3,bba03439a292a1e166f80264c16191cb,lmkebamcaaclubfxadlmueccxoimlema,1584,0,0,2010-03-30,2016-03-30,2010-03-30,2015-03-31,240.04,...,0.000000e+00,0.000000e+00,0,2192,365,2192,1584,0.000000e+00,1343.96,5
4,149d57cf92fc41cf94415803a877cb4b,MISSING,4425,0,526,2010-01-13,2016-03-07,2010-01-13,2015-03-09,445.75,...,2.896760e-06,4.860000e-10,0,2245,364,2245,-1887,0.000000e+00,3979.25,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14601,18463073fb097fc0ac5d3e040f356987,foosdfpfkusacimwkcsosbicdxkicaua,32270,47940,0,2012-05-24,2016-05-08,2015-05-08,2014-05-26,4648.01,...,0.000000e+00,0.000000e+00,0,1445,713,366,32270,1.485590e+00,27621.99,4
14602,d0a6f71671571ed83b2645d23af6de00,foosdfpfkusacimwkcsosbicdxkicaua,7223,0,181,2012-08-27,2016-08-27,2012-08-27,2015-08-28,631.69,...,5.684001e-03,2.987132e-04,1,1461,365,1461,5051,0.000000e+00,6591.31,4
14603,10e6828ddd62cbcf687cb74928c4c2d2,foosdfpfkusacimwkcsosbicdxkicaua,1844,0,179,2012-02-08,2016-02-07,2012-02-08,2015-02-09,190.39,...,2.896760e-06,4.860000e-10,1,1460,363,1460,-304,0.000000e+00,1653.61,4
14604,1cf20fd6206d7678d5bcafd28c53b4db,foosdfpfkusacimwkcsosbicdxkicaua,131,0,0,2012-08-30,2016-08-30,2012-08-30,2015-08-31,19.34,...,1.217891e-03,0.000000e+00,0,1461,365,1461,131,0.000000e+00,111.66,4
